https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#class_weights

https://medium.com/@zergtant/use-weighted-loss-function-to-solve-imbalanced-data-classification-problems-749237f38b75

Split ratio (TabNet): https://github.com/RAMitchell/GBM-Benchmarks/blob/master/benchmark.py


In [1]:
import sys
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

sys.path.append("../")

In [2]:
print(tf.__version__)

2.14.1


In [3]:
filepath = 'covtype.data.csv'
df = pd.read_csv(filepath, header=None)

columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
           'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
           'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
           'Soil_Type1' ,'Soil_Type2' ,'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 
           'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 
           'Soil_Type11' ,'Soil_Type12' ,'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 
           'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 
           'Soil_Type21' ,'Soil_Type22' ,'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 
           'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
           'Soil_Type31' ,'Soil_Type32' ,'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 
           'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
target = 'Cover_Type'

df.columns = columns
df[target] = df[target] - 1 # recode the integer values
df = df.astype(float) # convert all to float
print(df.shape)
df

(581012, 55)


Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0          2596.0    51.0    3.0                             258.0   
1          2590.0    56.0    2.0                             212.0   
2          2804.0   139.0    9.0                             268.0   
3          2785.0   155.0   18.0                             242.0   
4          2595.0    45.0    2.0                             153.0   
...           ...     ...    ...                               ...   
581007     2396.0   153.0   20.0                              85.0   
581008     2391.0   152.0   19.0                              67.0   
581009     2386.0   159.0   17.0                              60.0   
581010     2384.0   170.0   15.0                              60.0   
581011     2383.0   165.0   13.0                              60.0   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                                  0.0                            510.0   
1                                 -6.0                            390.0   
2                                 65.0                           3180.0   
3                                118.0                           3090.0   
4                                 -1.0                            391.0   
...                                ...                              ...   
581007                            17.0                            108.0   
581008                            12.0                             95.0   
581009                             7.0                             90.0   
581010                             5.0                             90.0   
581011                             4.0                             67.0   

        Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0               221.0           232.0          148.0   
1               220.0           235.0          151.0   
2               234.0           238.0          135.0   
3               238.0           238.0          122.0   
4               220.0           234.0          150.0   
...               ...             ...            ...   
581007          240.0           237.0          118.0   
581008          240.0           237.0          119.0   
581009          236.0           241.0          130.0   
581010          230.0           245.0          143.0   
581011          231.0           244.0          141.0   

        Horizontal_Distance_To_Fire_Points  ...  Soil_Type32  Soil_Type33  \
0                                   6279.0  ...          0.0          0.0   
1                                   6225.0  ...          0.0          0.0   
2                                   6121.0  ...          0.0          0.0   
3                                   6211.0  ...          0.0          0.0   
4                                   6172.0  ...          0.0          0.0   
...                                    ...  ...          ...          ...   
581007                               837.0  ...          0.0          0.0   
581008                               845.0  ...          0.0          0.0   
581009                               854.0  ...          0.0          0.0   
581010                               864.0  ...          0.0          0.0   
581011                               875.0  ...          0.0          0.0   

        Soil_Type34  Soil_Type35  Soil_Type36  Soil_Type37  Soil_Type38  \
0               0.0          0.0          0.0          0.0          0.0   
1               0.0          0.0          0.0          0.0          0.0   
2               0.0          0.0          0.0          0.0          0.0   
3               0.0          0.0          0.0          0.0          0.0   
4               0.0          0.0          0.0          0.0          0.0   
...             ...          ...          ...          ...          ...   
581007          0.0          0.0          0.0          0.0          0.0   
581008          0.0          0.0          0.0          0.0          

In [4]:
# This split is according to original work
'''
df_list = []
for c in range(0,len(df[target].unique())):
    df_list.append(df.loc[df[target] == c]) # separate rows according to their classes

random_state = 1
n_train = 1620
n_vald = 540
#tmp = df_list[0].sample(n=n_train, random_state=0)
#print(tmp)
np_train = np.empty((0,len(columns)))
np_vald = np.empty((0,len(columns)))
np_test = np.empty((0,len(columns)))
for df_ in df_list:
    df_tr = df_.sample(n=n_train, random_state=random_state) # randomly get training instances
    np_train = np.vstack((np_train, df_tr.values))
    #np_train.append(df_tr.values)
    df_ = df_.drop(index=df_tr.index) # drop training instances
    
    df_va = df_.sample(n=n_vald, random_state=random_state) # randomly get validation instances
    np_vald = np.vstack((np_vald, df_va.values))
    #np_vald.append(df_va.values)
    df_ = df_.drop(index=df_va.index) # drop validation instances
    
    np_test = np.vstack((np_test, df_.values))
    # np_test.append(df_) # the remaining is test instances   

y_train = np_train[:,-1]
y_vald = np_vald[:,-1]
y_test = np_test[:,-1]
X_train = np_train[:,:-1]
X_vald = np_vald[:,:-1]
X_test = np_test[:,:-1]
'''
from sklearn.model_selection import train_test_split

y = df[target].values.astype(np.float32)
X = df.drop(columns=[target]).values

# This split is according to Tab Survey (Borisov et al., 2022)
train_size = 0.7
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, train_size=train_size, random_state=0)
X_train, X_vald, y_train, y_vald = train_test_split(X_tmp, y_tmp, train_size=0.9, random_state=0)

print(f'Training set: {X_train.shape}')
print(f'Validation set: {X_vald.shape}')
print(f'Test set: {X_test.shape}')

Training set: (366037, 54)
Validation set: (40671, 54)
Test set: (174304, 54)


In [5]:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_vald, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0., 1., 2., 3., 4., 5., 6.], dtype=float32), array([18984, 25487,  3318,   255,   817,  1476,  1953], dtype=int64))
(array([0., 1., 2., 3., 4., 5., 6.], dtype=float32), array([2118, 2892,  348,   26,   89,  173,  165], dtype=int64))
(array([0., 1., 2., 3., 4., 5., 6.], dtype=float32), array([190738, 254922,  32088,   2466,   8587,  15718,  18392],
      dtype=int64))


Dataset is imbalanced. Calculate the weights for each class. The weights will be used during training to make the model pay more attention to under-represented classes

Let $w_c$ denotes the weight of class $c$. The weight is calculated as follows:

$w_c = \frac{1}{N_c} \times \frac{N}{C} = \frac{N}{N_c \times C}$

where $N$ is the total number of instances, $N_c$ is the number of instances for class $c$ and $C$ is the number of classes.

In [6]:
'''
num_target_instances = np.bincount(y_train.astype(np.int64))
N = np.sum(num_target_instances)
C = len(num_target_instances)

class_weight = {}
for c,s in enumerate(num_target_instances):
    w = (1 / s) * (N / C)
    class_weight[c] = w
print(class_weight)
'''

'\nnum_target_instances = np.bincount(y_train.astype(np.int64))\nN = np.sum(num_target_instances)\nC = len(num_target_instances)\n\nclass_weight = {}\nfor c,s in enumerate(num_target_instances):\n    w = (1 / s) * (N / C)\n    class_weight[c] = w\nprint(class_weight)\n'

In [12]:
def array_to_dataset(data, target, shuffle=True, batch_size=128):
    ds = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        ds = ds.shuffle(batch_size*2).batch(batch_size).prefetch(batch_size)
    else:
        ds = ds.batch(batch_size)
    return ds

batch_size = 1024
train_ds = array_to_dataset(X_train, y_train, batch_size=batch_size)
vald_ds = array_to_dataset(X_vald, y_vald, shuffle=False, batch_size=batch_size)
test_ds = array_to_dataset(X_test, y_test, shuffle=False, batch_size=batch_size)

In [10]:
## Multi-head/batch Attention
'''
b = 16
d = 32
inp = tf.random.uniform((b, d)) # (batch, n_features)
print(inp.shape)

c = 3
h = 10
w = tf.random.uniform((h, d, c)) # (head, n_features, class)
print(w.shape)

z = tf.matmul(inp, w) # (batch, n_features) dot (head, n_features, class) = (head, batch, class)
print(z.shape)

z = tf.nn.softmax(z, axis=-1)
print(z.shape)

r = 5.24
w = tf.math.exp(w * r) # amplify weights; (head, n_features, class)
w_ = tf.transpose(w, perm=(0, 2, 1))
print(w_.shape) # (head, class, n_features)

a = tf.matmul(z, w_) # (head, batch, class) dot (head, class, n_features) = (head, batch, n_features)
print(a.shape) # (head, batch, n_features)
#a = tf.reduce_mean(a, axis=1) # (head, n_features)
print(a.shape)



## Multi-head IFE
input_data = tf.random.normal((b, d))
print(f'input shape: {input_data.shape}')
input_scores = [tf.random.normal((h, b, d, 1)) for j in range(0,d)]
input_scores = tf.concat(input_scores, axis=-1) # shape = (head, batch, n_features, n_features)
print(f'score shape: {input_scores.shape}')
#input_scores = tf.reduce_mean(input_scores, axis=[-1, 0]) # shape = (batch, n_features)
#input_scores = tf.nn.softmax(input_scores, axis=-1) # shape = (batch, n_features)
#print(f'score shape: {input_scores.shape}')

# Instead of calculating mean across dimension -1 (n_features) and 0 (head), calculate mean across n_features only
input_scores = tf.reduce_mean(input_scores, axis=[-1]) # shape = (head, batch, n_features)
print(f'score shape: {input_scores.shape}')

# input_data = tf.expand_dims(input_data, axis=0)
input_data = tf.broadcast_to(input_data, [h, b, d])
print(f'input shape: {input_data.shape}')

weighted_inputs = input_data * input_scores
#weighted_inputs = tf.reduce_mean(weighted_inputs, axis=[0])
#print(f'weighted_inputs shape: {weighted_inputs.shape}')

x = tf.transpose(weighted_inputs, perm=(1,0,2))
print(x.shape)
x = tf.reshape(x, shape=(b, -1))
print(x.shape)
x = tf.keras.layers.Dense(units=32)(x)
print(x.shape)
'''

"\nb = 16\nd = 32\ninp = tf.random.uniform((b, d)) # (batch, n_features)\nprint(inp.shape)\n\nc = 3\nh = 10\nw = tf.random.uniform((h, d, c)) # (head, n_features, class)\nprint(w.shape)\n\nz = tf.matmul(inp, w) # (batch, n_features) dot (head, n_features, class) = (head, batch, class)\nprint(z.shape)\n\nz = tf.nn.softmax(z, axis=-1)\nprint(z.shape)\n\nr = 5.24\nw = tf.math.exp(w * r) # amplify weights; (head, n_features, class)\nw_ = tf.transpose(w, perm=(0, 2, 1))\nprint(w_.shape) # (head, class, n_features)\n\na = tf.matmul(z, w_) # (head, batch, class) dot (head, class, n_features) = (head, batch, n_features)\nprint(a.shape) # (head, batch, n_features)\n#a = tf.reduce_mean(a, axis=1) # (head, n_features)\nprint(a.shape)\n\n\n\n## Multi-head IFE\ninput_data = tf.random.normal((b, d))\nprint(f'input shape: {input_data.shape}')\ninput_scores = [tf.random.normal((h, b, d, 1)) for j in range(0,d)]\ninput_scores = tf.concat(input_scores, axis=-1) # shape = (head, batch, n_features, n_feat

In [12]:
from ife as IFENetClassifier

n_features = X_train.shape[1]
_, counts = np.unique(y_train, return_counts=True)
n_classes = len(counts)
ife_num_layers = 1
clf_num_layers = 1
clf_hidden_units = [64]
reduction_layer = 'average'
num_att = 8
r = 5.9675

print(f'n_classes: {n_classes}')
print(f'n_features: {n_features}')

ife_params = {'n_features': n_features,
              'n_outputs': n_classes,
              'num_att': num_att,
              'r': r,
              'ife_num_layers': ife_num_layers, 
              'clf_num_layers': clf_num_layers,
              'clf_hidden_units': clf_hidden_units,
              'reduction_layer': reduction_layer
             }
model = IFENetClassifier(**ife_params)
# model = model.build(input_shape=(n_features,))

input_data = tf.random.normal((batch_size, n_features))
model(input_data)

n_classes: 7
n_features: 54


<tf.Tensor: shape=(15000, 7), dtype=float32, numpy=
array([[0.14012766, 0.13991402, 0.14376368, ..., 0.1451238 , 0.14457028,
        0.14098172],
       [0.14287397, 0.14226663, 0.14319076, ..., 0.14439136, 0.14164281,
        0.14390671],
       [0.1428167 , 0.14375235, 0.1408631 , ..., 0.13957396, 0.14396857,
        0.14332812],
       ...,
       [0.14201196, 0.14170015, 0.14253895, ..., 0.1416966 , 0.14398953,
        0.1450189 ],
       [0.14372541, 0.13655058, 0.14240709, ..., 0.14182338, 0.14742263,
        0.14422187],
       [0.14277346, 0.13976255, 0.14221719, ..., 0.14502841, 0.14626305,
        0.14122474]], dtype=float32)>

In [13]:
x = tf.keras.Input(shape=(n_features,))
# x = tf.keras.layers.Input(shape=input_shape)
model = tf.keras.models.Model(inputs=[x], outputs=model.call(x))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 54)]                 0         []                            
                                                                                                  
 tf.compat.v1.shape (TFOpLa  (2,)                         0         ['input_1[0][0]']             
 mbda)                                                                                            
                                                                                                  
 preprocess_batch_norm (Bat  (None, 54)                   216       ['input_1[0][0]']             
 chNormalization)                                                                                 
                                                                                              

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

lr = 0.005
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, 
                                                              decay_steps=2000,
                                                              decay_rate=0.95,
                                                              staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

checkpoint_path = 'checkpoints/ifeNet_cover.h5'
patience = 100
callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss'),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, monitor='val_accuracy')]

epochs = 5
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [ ]:
saved_model_path = 'saved_model/ifeNet_cover.h5'
model.fit(train_ds, validation_data=vald_ds, epochs=epochs, callbacks=callbacks)
model.load_weights(checkpoint_path)
model.save_weights(saved_model_path)

In [8]:
# Best model

from ife import IFENetClassifier

n_features = X_train.shape[1]
_, counts = np.unique(y_train, return_counts=True)
n_classes = len(counts)
ife_num_layers = 1
clf_num_layers = 1
clf_hidden_units = [115]
reduction_layer = 'flatten'
num_att = 75
r = 4.15

print(f'n_classes: {n_classes}')
print(f'n_features: {n_features}')

ife_params = {'n_features': n_features,
              'n_outputs': n_classes,
              'num_att': num_att,
              'r': r,
              'ife_num_layers': ife_num_layers, 
              'clf_num_layers': clf_num_layers,
              'clf_hidden_units': clf_hidden_units,
              'reduction_layer': reduction_layer
             }
model = IFENetClassifier(**ife_params)

model.build(input_shape=(None,n_features,))
#model.summary()

path_saved_model = 'saved_model/ifeNet_cover_24.h5'
model.load_weights(path_saved_model)

n_classes: 7
n_features: 54


In [ ]:
y_pred = np.empty((0,))
y_test = np.empty((0,))

for data,label in test_ds.take(2):
    y_hat = model(data)
    y_hat = np.argmax(y_hat, axis=-1)
    y_pred = np.append(y_pred, y_hat.ravel())

    label = label.numpy()
    y_test = np.append(y_test, label.ravel())

In [ ]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [55]:
first_batch = None
for i, batch in enumerate(test_ds):
    if i == 0:  # Indexing starts at 0, so the second batch has index 1
        first_batch = batch
        break  # Once the second batch is found, break the loop

data, label = first_batch
model(data)

feat_scores = model.input_scores
feat_scores = np.mean(feat_scores, axis=(0,1))

feat_rank = {}
for col,score in zip(columns,feat_scores):
    #print(f'{col}: {score}')
    feat_rank[col] = score

df_feat_rank = pd.DataFrame(list(feat_rank.items()), columns=['Feature', 'Score'])
df_feat_rank.sort_values(by='Score', ascending=False)

Feature     Score
0                            Elevation  0.127220
20                          Soil_Type7  0.060129
50                         Soil_Type37  0.057631
28                         Soil_Type15  0.044712
21                          Soil_Type8  0.039093
10                    Wilderness_Area1  0.035264
5      Horizontal_Distance_To_Roadways  0.029630
7                       Hillshade_Noon  0.029422
6                        Hillshade_9am  0.026024
49                         Soil_Type36  0.024337
2                                Slope  0.023465
31                         Soil_Type18  0.021535
38                         Soil_Type25  0.019394
48                         Soil_Type35  0.018082
8                        Hillshade_3pm  0.018062
3     Horizontal_Distance_To_Hydrology  0.018017
12                    Wilderness_Area3  0.017555
14                          Soil_Type1  0.017503
34                         Soil_Type21  0.016906
47                         Soil_Type34  0.016599
51                         Soil_Type38  0.015525
9   Horizontal_Distance_To_Fire_Points  0.014728
40                         Soil_Type27  0.014447
41                         Soil_Type28  0.013442
4       Vertical_Distance_To_Hydrology  0.013118
45                         Soil_Type32  0.013044
35                         Soil_Type22  0.012877
33                         Soil_Type20  0.011980
44                         Soil_Type31  0.011614
30                         Soil_Type17  0.011607
22                          Soil_Type9  0.011441
1                               Aspect  0.010871
11                    Wilderness_Area2  0.010731
19                          Soil_Type6  0.010589
18                          Soil_Type5  0.010118
16                          Soil_Type3  0.009811
46                         Soil_Type33  0.009791
26                         Soil_Type13  0.009780
43                         Soil_Type30  0.009470
42                         Soil_Type29  0.009350
52                         Soil_Type39  0.009291
36                         Soil_Type23  0.008780
37                         Soil_Type24  0.008220
15                          Soil_Type2  0.008210
27                         Soil_Type14  0.007813
32                         Soil_Type19  0.007771
53                         Soil_Type40  0.007762
17                          Soil_Type4  0.007631
23                         Soil_Type10  0.007594
13                    Wilderness_Area4  0.007238
24                         Soil_Type11  0.006812
25                         Soil_Type12  0.006083
39                         Soil_Type26  0.006080
29                         Soil_Type16  0.005778

In [56]:
second_batch = None
for i, batch in enumerate(test_ds):
    if i == 1:  # Indexing starts at 0, so the second batch has index 1
        second_batch = batch
        break  # Once the second batch is found, break the loop

data,label = second_batch
model(data)

feat_scores = model.input_scores
feat_scores = np.mean(feat_scores, axis=(0,1))

feat_rank = {}
for col,score in zip(columns,feat_scores):
    #print(f'{col}: {score}')
    feat_rank[col] = score

df_feat_rank = pd.DataFrame(list(feat_rank.items()), columns=['Feature', 'Score'])
df_feat_rank.sort_values(by='Score', ascending=False)

Feature     Score
0                            Elevation  0.131297
20                          Soil_Type7  0.060262
50                         Soil_Type37  0.057748
28                         Soil_Type15  0.044375
21                          Soil_Type8  0.037752
10                    Wilderness_Area1  0.036080
5      Horizontal_Distance_To_Roadways  0.029566
7                       Hillshade_Noon  0.029403
49                         Soil_Type36  0.024609
6                        Hillshade_9am  0.023704
2                                Slope  0.022405
31                         Soil_Type18  0.021359
8                        Hillshade_3pm  0.019857
38                         Soil_Type25  0.019659
48                         Soil_Type35  0.017578
14                          Soil_Type1  0.017577
12                    Wilderness_Area3  0.017233
3     Horizontal_Distance_To_Hydrology  0.017106
34                         Soil_Type21  0.016443
47                         Soil_Type34  0.015728
51                         Soil_Type38  0.015290
9   Horizontal_Distance_To_Fire_Points  0.015106
40                         Soil_Type27  0.014788
41                         Soil_Type28  0.013800
45                         Soil_Type32  0.013036
4       Vertical_Distance_To_Hydrology  0.012701
33                         Soil_Type20  0.012394
35                         Soil_Type22  0.012279
44                         Soil_Type31  0.011925
22                          Soil_Type9  0.011894
1                               Aspect  0.011474
30                         Soil_Type17  0.011389
18                          Soil_Type5  0.010481
19                          Soil_Type6  0.010376
11                    Wilderness_Area2  0.010288
42                         Soil_Type29  0.009861
26                         Soil_Type13  0.009735
43                         Soil_Type30  0.009285
46                         Soil_Type33  0.009279
16                          Soil_Type3  0.009095
52                         Soil_Type39  0.009072
36                         Soil_Type23  0.008844
37                         Soil_Type24  0.008769
15                          Soil_Type2  0.008395
17                          Soil_Type4  0.007836
27                         Soil_Type14  0.007833
32                         Soil_Type19  0.007809
53                         Soil_Type40  0.007682
23                         Soil_Type10  0.007225
13                    Wilderness_Area4  0.007099
24                         Soil_Type11  0.006998
25                         Soil_Type12  0.006314
39                         Soil_Type26  0.006019
29                         Soil_Type16  0.005870